In [1]:
import os

In [2]:
from langchain.document_loaders import PyPDFLoader

In [3]:
%pwd

'd:\\interview_qa\\research'

In [4]:
%cd ..

d:\interview_qa


In [5]:
file_path = "D:\interview_qa\Machine - Learning - Tom Mitchell.pdf"
loader = PyPDFLoader(file_path)
data = loader.load()

<>:1: SyntaxWarning: invalid escape sequence '\i'
<>:1: SyntaxWarning: invalid escape sequence '\i'
C:\Users\himan\AppData\Local\Temp\ipykernel_9596\804336038.py:1: SyntaxWarning: invalid escape sequence '\i'
  file_path = "D:\interview_qa\Machine - Learning - Tom Mitchell.pdf"
C:\Users\himan\AppData\Local\Temp\ipykernel_9596\804336038.py:1: SyntaxWarning: invalid escape sequence '\i'
  file_path = "D:\interview_qa\Machine - Learning - Tom Mitchell.pdf"


KeyboardInterrupt: 

In [ ]:
data

[Document(metadata={'source': 'D:\\interview_qa\\Machine - Learning - Tom Mitchell.pdf', 'page': 0}, page_content=''),
 Document(metadata={'source': 'D:\\interview_qa\\Machine - Learning - Tom Mitchell.pdf', 'page': 1}, page_content=' \n \nMachine Learning \n \nTom M. Mitchell \n \n \nProduct Details  \n• Hardcover: 432 pages ; Dimensions (in inches): 0.75 x 10.00 x 6.50  \n• Publisher: McGraw-Hill Science/Engineering/Math; (March 1, 1997)  \n• ISBN: 0070428077  \n• Average Customer Review: \n Based on 16 reviews. \n• Amazon.com Sales Rank: 42,816  \n• Popular in: Redmond, WA (#17) , Ithaca, NY (#9)  \n \n \nEditorial Reviews \nFrom Book News, Inc. An introductory text on primary approaches to machine learning and \nthe study of computer algorithms that improve  automatically through experience. Introduce \nbasics concepts from statistics, artificial intellig ence, information theory, and other disciplines as \nneed arises, with balanced coverage of theory and practice, and presents ma

In [ ]:
len(data)

421

In [ ]:
question_gen = ""
for page in data:
    question_gen += page.page_content

In [ ]:
len(question_gen)

1094558

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100000000, chunk_overlap=100)

In [ ]:
splits = text_splitter.split_text(question_gen[:10000])

In [ ]:
documents = [Document(page_content=split) for split in splits]

In [ ]:
len(documents)

1

In [ ]:
type(documents[0])

langchain_core.documents.base.Document

In [ ]:
text_splitter1 = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

In [ ]:
document_answer_gen = text_splitter1.split_documents(
    documents
)

In [ ]:
len(document_answer_gen)

12

In [ ]:
%pip install langchain_ollama

  Using cached langchain_ollama-0.2.0-py3-none-any.whl.metadata (1.8 kB)
Using cached langchain_ollama-0.2.0-py3-none-any.whl (14 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model = "llama3.2:1b",
    temperature = 0.8,
    num_predict = 256,
    # other params ...
)

In [ ]:
prompt_template = """
You are an expert at creating questions based on coding materials and documentation.
Your goal is to prepare a coder or programmer for their exam and coding tests.
You do this by asking questions about the text below:

------------
{text}
------------

Create questions that will prepare the coders or programmers for their tests.
Make sure not to lose any important information.
There should be only 10 questions in output without answer and nothing else not even your output.

QUESTIONS:
"""

In [ ]:
from langchain.prompts import PromptTemplate

In [ ]:
PROMPT_QUESTIONS = PromptTemplate(template=prompt_template, input_variables=['text'])

In [ ]:
refine_template = ("""
You are an expert at creating practice questions based on coding material and documentation.
Your goal is to help a coder or programmer prepare for a coding test.
We have received some practice questions to a certain extent: {existing_answer}.
We have the option to refine the existing questions or add new ones.
(only if necessary) with some more context below.
------------
{text}
------------

Given the new context, refine the original questions in English.
If the context is not helpful, please provide the original questions.
QUESTIONS:
"""
)


In [ ]:
REFINE_PROMPT_QUESTIONS = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)

In [ ]:
from langchain.chains.summarize import load_summarize_chain

In [ ]:
ques_gen_chain = load_summarize_chain(llm = llm, 
                                          chain_type = "refine", 
                                          verbose = True, 
                                          question_prompt=PROMPT_QUESTIONS, 
                                          refine_prompt=REFINE_PROMPT_QUESTIONS)

In [ ]:
ques = ques_gen_chain.invoke(documents)

print(ques)



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert at creating questions based on coding materials and documentation.
Your goal is to prepare a coder or programmer for their exam and coding tests.
You do this by asking questions about the text below:

------------
Machine Learning 
 
Tom M. Mitchell 
 
 
Product Details  
• Hardcover: 432 pages ; Dimensions (in inches): 0.75 x 10.00 x 6.50  
• Publisher: McGraw-Hill Science/Engineering/Math; (March 1, 1997)  
• ISBN: 0070428077  
• Average Customer Review: 
 Based on 16 reviews. 
• Amazon.com Sales Rank: 42,816  
• Popular in: Redmond, WA (#17) , Ithaca, NY (#9)  
 
 
Editorial Reviews 
From Book News, Inc. An introductory text on primary approaches to machine learning and 
the study of computer algorithms that improve  automatically through experience. Introduce 
basics concepts from statistics, artificial intellig ence, information theory, and other discipline

In [ ]:
result = ques['output_text']

In [ ]:
print(result)

Here are 10 questions based on the text:

1. What is the field of study that this book belongs to?
2. Who has helped write chapter drafts for this book?
3. What have computers been able to learn about people in recent years?
4. What type of learning tasks have algorithms been effective at solving?
5. Why are machine learning outperforming other approaches in speech recognition?
6. What field is known as the "field where humans do not yet know how to make computers learn nearly as well as people learn"?
7. What types of problems have been successfully solved using computer programs that exhibit useful forms of learning?
8. What was a major challenge when developing algorithms for machine learning?
9. How has technology advanced in recent years in terms of making computers learn better than humans?
10. What is the current state of understanding regarding human learning abilities and disabilities as compared to computer learning capabilities?


In [ ]:
from langchain_ollama import OllamaEmbeddings

embed = OllamaEmbeddings(
    model="llama3.2:1b"
)

In [ ]:
from langchain.vectorstores import FAISS

In [ ]:
vector_store = FAISS.from_documents(document_answer_gen, embed)

In [ ]:
llm_answer_gen =ChatOllama(
    model = "llama3.2:1b",
    temperature = 0.8,
    num_predict = 256,
    # other params ...
)

In [ ]:
result_list = result.split("\n")

In [ ]:
result = result_list[2:12]
    

In [ ]:
result

['1. What is being discussed in this passage about the potential of computers to learn?',
 '2. Who has contributed to creating online resources, including Jason Rennie and Paul Hsiung.',
 '3. What type of learning tasks have algorithms been invented to be effective at?',
 '4. How do speech recognition programs outperform other approaches for certain problems?',
 '5. What is the focus of the theoretical understanding of human learning abilities (and disabilities) being discussed in this passage?',
 '6. What type of computer program has begun to appear as a commercial application due to its effectiveness in certain tasks.',
 '7. What field is known for algorithms based on machine learning?',
 '8. What problems have algorithms based on machine learning been effective at solving so far?',
 '9. Who has helped to create online resources, including those mentioned earlier (Jason Rennie and Paul Hsiung).',
 '10. How would you describe the potential of computers to learn nearly as well as peopl

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
answer_generation_chain = RetrievalQA.from_chain_type(llm=llm_answer_gen, 
                                               chain_type="stuff", 
                                               retriever=vector_store.as_retriever())


In [ ]:
# Answer each question and save to a file
for question in result:
    print("Question: ", question)
    answer = answer_generation_chain.run(question)
    print("Answer: ", answer)
    print("\n")
    # Save answer to file
    with open("answers1.txt", "a") as f:
        f.write("Question: " + question + "\n")
        f.write("Answer: " + answer + "\n"+ "\n"+ "\n")
       




Question:  1. What is being discussed in this passage about the potential of computers to learn?
Answer:  The passage is discussing the potential of computers to learn and perform tasks autonomously, similar to humans. It mentions that there are several researchers, including Drew Ng, Joseph O'Sullivan, Patrawadee Prasangsit, Doina Precup, Bob Price, Choon Quek, Sean Slattery, Belinda Thom, Astro Teller, Will Tracz, Joan Mitchell, and others, who have contributed to the development of this area.


Question:  2. Who has contributed to creating online resources, including Jason Rennie and Paul Hsiung.
Answer:  According to the text, it appears that the following individuals have contributed to creating online resources related to machine learning:

1. Jason Rennie
2. Paul Hsiung


Question:  3. What type of learning tasks have algorithms been invented to be effective at?
Answer:  According to the text, machine learning algorithms have been invented to be effective at a variety of learnin